In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import timm
import torch

from augmentations import Masking, GaussianNoise, Stretch
from data import get_datasets
from AdaMatch import AdaMatch
from Baseline import Baseline
from FixMatch import FixMatch
from model import Model
from metrics import AUC, Error_Rate

In [2]:
# Hyperparameters
epochs = 4096
learning_rate = 1e-3
momentum = 0.9
mu = 7
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Data
train_dataset, unlabeled_dataset, test_dataset = get_datasets('./data', name='cifar10', download=True, transform=transforms.ToTensor(), n_labeled=64, seed=4)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size*mu, shuffle=True)
if len(np.unique([np.argmax(x[1]) for x in train_dataset]))==10: print('All labels in the training set')

All labels in the training set


In [3]:
weak = transforms.Compose([
    transforms.RandomHorizontalFlip()
])

strong = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomAutocontrast(),
    transforms.GaussianBlur(kernel_size=(5,9), sigma=(.1, 2.)),
    transforms.RandomAffine(80, translate=(.3, .3), scale=(.8, 1.2), shear=(-.1, .1, -.1, .1)),
    transforms.RandomErasing()
])

### Augmentation

In [8]:
config = {'backbone':'resnet18','n_channels':3, 'pretrained':False}
model = Model(10, config=config)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, eta_min=1e-8)
criterion = nn.functional.cross_entropy
metrics = [AUC,Error_Rate]

trainer = Baseline(weak_transform=weak)
model, log_baseline = trainer.train(train_loader, test_loader, model, optimizer, criterion, metrics, scheduler=scheduler, epochs=epochs)

Epoch 4096/4096
 Training 	|	 loss=0.005
 Validation 	|	 loss=812.158  -  AUC=0.568  -  Error_Rate=0.85

 Best : AUC=0.598  -  Error_Rate=0.866 at epoch 64


### FixMatch

In [9]:
config = {'backbone':'resnet18', 'n_channels':3}
model = Model(10, config=config)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, eta_min=1e-8)
criterion = nn.functional.cross_entropy
metrics = [AUC,Error_Rate]


trainer = FixMatch(tau=0.95, strong_transform=strong, weak_transform=weak)
model, log_fixmatch = trainer.train(train_loader, unlabeled_loader, test_loader, model, optimizer, criterion, metrics, scheduler=scheduler, epochs=epochs)

Epoch 4096/4096
 Training 	|	 loss=[0.196 0.01  0.186]  |  49.0 Pseudo-labels
 Validation 	|	 loss=631.61  -  AUC=0.603  -  Error_Rate=0.822

 Best : AUC=0.609  -  Error_Rate=0.85 at epoch 76


### AdaMatch

In [4]:
config = {'backbone':'resnet18', 'n_channels':3}
model = Model(10, config=config)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, eta_min=1e-8)
criterion = nn.functional.cross_entropy
metrics = [AUC,Error_Rate]


trainer = AdaMatch(tau=0.95, strong_transform=strong, weak_transform=weak)
model, log_fixmatch = trainer.train(train_loader, unlabeled_loader, test_loader, model, optimizer, criterion, metrics, scheduler=scheduler, epochs=epochs)

Epoch 4096/4096
 Training 	|	 loss=[2.763 1.937 0.826]  |  238.0 Pseudo-labels
 Validation 	|	 loss=438.378  -  AUC=0.682  -  Error_Rate=0.787

 Best : AUC=0.606  -  Error_Rate=0.844 at epoch 120
